In [1]:
# Importing required libraries
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications.inception_v3 import preprocess_input
import numpy as np

In [2]:
# File paths
train_dir = "C:/Users/jssri/Downloads/Dataset/dataset/train"
validation_dir = "C:/Users/jssri/Downloads/Dataset/dataset/validation"

In [3]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32
target_size = (299, 299)  # InceptionV3 requires 299x299 input size

In [4]:
# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

train_generator[0][0]


Found 16 images belonging to 2 classes.
Found 4 images belonging to 2 classes.


array([[[[9.6103030e-01, 6.1325324e-01, 0.0000000e+00],
         [9.7584420e-01, 6.3062513e-01, 0.0000000e+00],
         [9.8466581e-01, 6.4223498e-01, 0.0000000e+00],
         ...,
         [9.8431379e-01, 1.0000000e+00, 9.9607849e-01],
         [9.8431379e-01, 1.0000000e+00, 9.9607849e-01],
         [9.8431379e-01, 1.0000000e+00, 9.9607849e-01]],

        [[9.6416324e-01, 6.1787665e-01, 6.4585358e-04],
         [9.6839887e-01, 6.2319773e-01, 0.0000000e+00],
         [9.7484088e-01, 6.2985307e-01, 0.0000000e+00],
         ...,
         [9.8431379e-01, 1.0000000e+00, 9.9607849e-01],
         [9.8431379e-01, 1.0000000e+00, 9.9607849e-01],
         [9.8431379e-01, 1.0000000e+00, 9.9607849e-01]],

        [[9.5555955e-01, 6.0957563e-01, 1.9988283e-03],
         [9.5210707e-01, 6.0700905e-01, 0.0000000e+00],
         [9.5691270e-01, 6.1336368e-01, 0.0000000e+00],
         ...,
         [9.8431379e-01, 1.0000000e+00, 9.9607849e-01],
         [9.8431379e-01, 1.0000000e+00, 9.9607849e-01],
  

In [5]:
# To view target values, it is automatically one-hot encoded
train_generator[0][1]

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [6]:
# InceptionV3 model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))



In [7]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Own customization
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dense(train_generator.num_classes, activation='softmax')
])

In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [9]:
epochs = 5
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

Epoch 1/5


1/1 [==============================] - 9s 9s/step - loss: 0.4987 - accuracy: 0.8125 - val_loss: 0.0572 - val_accuracy: 1.0000
Epoch 2/5
1/1 [==============================] - 1s 1s/step - loss: 0.0395 - accuracy: 1.0000 - val_loss: 0.0105 - val_accuracy: 1.0000
Epoch 3/5
1/1 [==============================] - 1s 1s/step - loss: 0.0038 - accuracy: 1.0000 - val_loss: 0.0031 - val_accuracy: 1.0000
Epoch 4/5
1/1 [==============================] - 1s 1s/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 5/5
1/1 [==============================] - 1s 1s/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 6.8135e-04 - val_accuracy: 1.0000


In [10]:
# Evaluate
loss, accuracy = model.evaluate(validation_generator)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)

1/1 [==============================] - 0s 314ms/step - loss: 6.8135e-04 - accuracy: 1.0000
Validation Loss: 0.0006813482032157481
Validation Accuracy: 1.0


In [11]:
# Preprocess new user input
def predict_image(model, img_path):
    img = load_img(img_path, target_size=(299, 299))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    return preds

In [12]:
# New user input
img_path = "C:/Users/jssri/Downloads/Dataset/dataset/test/2p.png"
predictions = predict_image(model, img_path)
print(predictions)

# Assuming class indices are 0 for "Burger" and 1 for "Pizza"
print("Burger" if np.argmax(predictions[0]) == 0 else "Pizza")

1/1 [==============================] - 2s 2s/step
[[0.0024023  0.99759775]]
Pizza
